In [1]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh
%cd ..

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 4.45 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 27.0 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2025-07-13 00:53:20--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.142.25, 104.192.142.26, 104.192.142.24, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.142.25|:443... connected.
HTTP request sent, a

In [2]:
!pip install hgtk

In [3]:
# fasttext 설 치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .
%cd ..

Cloning into 'fastText'...
remote: Enumerating objects: 3998, done.
remote: Counting objects: 100% (995/995), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 3998 (delta 886), reused 824 (delta 824), pack-reused 3003 (from 2)
Receiving objects: 100% (3998/3998), 8.30 MiB | 10.23 MiB/s, done.
Resolving deltas: 100% (2527/2527), done.
/content/fastText
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++17 -m

In [4]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt",filename="ratings_total.txt")


('ratings_total.txt', <http.client.HTTPMessage at 0x7ed4b4271750>)

In [6]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전 체 리 뷰 개 수 :',len(total_data)) # 전 체 리 뷰 개 수 출 력

전 체 리 뷰 개 수 : 200000


In [7]:
total_data[:5]


,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [8]:
# 한 글 인 지 체 크
hgtk.checker.is_hangul('ㄱ')

True

In [9]:
# 한 글 인 지 체 크
hgtk.checker.is_hangul('28')


False

In [10]:
# 음 절 을 초 성 , 중 성 , 종 성 으 로 분 해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [11]:
# 초 성 , 중 성 을 결 합
hgtk.letter.compose('ㄴ', 'ㅏ')


'나'

In [12]:
# 초 성 , 중 성 , 종 성 을 결 합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

In [13]:
# 한 글 이 아 닌 입 력 에 대 해 서 는 에 러 발 생.
hgtk.letter.decompose('1')

NotHangulException: 

In [14]:
# 결 합 할 수 없 는 상 황 에 서 는 에 러 발 생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: No valid Hangul character index

In [15]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음 절)을 초 성 , 중 성 , 종 성 으 로 분 리
      cho, jung, jong = hgtk.letter.decompose(char)
      # 자 모 가 빈 문 자 일 경 우 특 수 문 자 -로 대 체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong
      # 만 약 char(음 절)이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char
  # 단 어 토 큰 의 자 모 단 위 분 리 결 과 를 추 가
  return decomposed_token

In [16]:
word_to_jamo('남 동 생')


'ㄴㅏㅁ ㄷㅗㅇ ㅅㅐㅇ'

In [17]:
word_to_jamo('여 동 생')


'ㅇㅕ- ㄷㅗㅇ ㅅㅐㅇ'

In [20]:
mecab = Mecab()
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만와서 당황했습니다.'))


['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [22]:
def tokenize_by_jamo(s):
  return [word_to_jamo(token) for token in mecab.morphs(s)]

In [23]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만와서 당황했습니다.'))


['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [24]:
tokenized_data = []
for sample in tqdm(total_data['reviews'].to_list()):
  tokenzied_sample = tokenize_by_jamo(sample) # 자 소 단 위 토 큰 화
  tokenized_data.append(tokenzied_sample)


100%|██████████| 200000/200000 [01:21<00:00, 2454.32it/s]


In [25]:
print(tokenized_data[0])

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']


In [29]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0
  # 1. 초 기 입 력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'
  while index < len(jamo_sequence):
    # 문 자 가 한 글(정 상 적 인 자 모)이 아 닐 경 우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1
    # 문 자 가 정 상 적 인 자 모 라 면 초 성 , 중 성 , 종 성 을 하 나 의 토 큰 으 로 간 주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3
  # 2. 자 모 단 위 토 큰 화 완 료
  # tokenized_jamo : ['ㄴ ㅏ ㅁ', 'ㄷ ㅗ ㅇ', 'ㅅ ㅐ ㅇ']
  word = ''
  try:
    for jamo in tokenized_jamo:
      # 초 성 , 중 성 , 종 성 의 묶 음 으 로 추 정 되 는 경 우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종 성 이 존 재 하 지 않 는 경 우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종 성 이 존 재 하 는 경 우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한 글 이 아 닌 경 우
      else:
        word = word + jamo
  # 복 원 중(hgtk.letter.compose) 에 러 발 생 시 초 기 입 력 리 턴.
  # 복 원 이 불 가 능 한 경 우 예 시) 'ㄴ!ㅁ ㄷ ㅗ ㅇ ㅅ ㅐ ㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence
  # 3. 단 어 로 복 원 완 료
  # word : '남 동 생'
  return word

In [30]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

In [31]:
import fasttext


In [32]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')


100%|██████████| 200000/200000 [00:01<00:00, 120145.02 line/s]


In [33]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')
model.save_model("fasttext.bin") # 모 델 저 장
model = fasttext.load_model("fasttext.bin") # 모 델 로 드

In [34]:
model[word_to_jamo('남동생')]

array([-5.12988716e-02,  6.32482350e-01,  1.86406955e-01,  3.02682549e-01,
        7.58056223e-01, -4.81922895e-01,  7.69727588e-01, -4.57956076e-01,
       -2.45553285e-01, -1.62321270e-01,  2.81749815e-01,  2.63079137e-01,
       -7.32175648e-01, -7.20778573e-03,  5.17405629e-01, -1.58245549e-01,
        2.95751601e-01,  2.11942822e-01, -6.13275826e-01, -5.24447501e-01,
       -1.22135669e-01, -8.72220576e-01,  8.69588077e-01, -2.17704460e-01,
       -1.02253520e+00, -5.55667758e-01,  8.56216490e-01,  1.88477576e-01,
        3.47477168e-01,  1.09434962e+00,  4.50499713e-01, -5.51607609e-02,
       -3.12900320e-02,  4.84812349e-01,  5.57484388e-01, -2.87604272e-01,
        3.42785805e-01, -4.73892927e-01,  7.12700794e-03, -2.27341160e-01,
        8.31643045e-01,  6.99855268e-01, -4.87740606e-01,  9.92074847e-01,
       -6.09196424e-02,  8.26852024e-02,  8.90996695e-01, -1.05027795e-01,
       -7.22995102e-01, -3.50981772e-01,  1.57418400e-01,  3.88410956e-01,
       -1.48920923e-01,  

In [35]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8895309567451477, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8558163642883301, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7989221215248108, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7835164666175842, 'ㅊㅣㄴㄱㅜ-'),
 (0.7683124542236328, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7323827147483826, 'ㅈㅗ-ㅋㅏ-'),
 (0.7193372845649719, 'ㄴㅏㅁㅇㅏ-'),
 (0.7170660495758057, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.7109101414680481, 'ㅎㅏㄱㅅㅐㅇ'),
 (0.6993108987808228, 'ㅇㅓㄴㄴㅣ-')]

In [36]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]


In [37]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))


[('동생', 0.8895309567451477), ('남친', 0.8558163642883301), ('남편', 0.7989221215248108), ('친구', 0.7835164666175842), ('생일', 0.7683124542236328), ('조카', 0.7323827147483826), ('남아', 0.7193372845649719), ('중학생', 0.7170660495758057), ('학생', 0.7109101414680481), ('언니', 0.6993108987808228)]


In [38]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10)))


[('남동생', 0.9027712345123291), ('남친', 0.8256592154502869), ('남매', 0.7731515765190125), ('남짓', 0.7305405139923096), ('남아', 0.7102065086364746), ('남녀', 0.7093713879585266), ('남김', 0.6990870833396912), ('남긴', 0.691932737827301), ('남여', 0.6827714443206787), ('남편', 0.6807946562767029)]


In [39]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동셍 ㅋ'), k=10)))

[('남동생', 0.8049336075782776), ('남친', 0.7445354461669922), ('남녀', 0.6474481821060181), ('남짓', 0.6361883282661438), ('남아', 0.6326759457588196), ('남김', 0.629916787147522), ('남여', 0.6200527548789978), ('동생', 0.6051467061042786), ('남매', 0.6047568917274475), ('남길', 0.6015027165412903)]


In [40]:
print(transform(model.get_nearest_neighbors(word_to_jamo('난동생'), k=10)))


[('남동생', 0.8670289516448975), ('난생', 0.8337627649307251), ('남편', 0.8116278648376465), ('동생', 0.781231164932251), ('남친', 0.7788612246513367), ('남아', 0.7372268438339233), ('중학생', 0.7353932857513428), ('남매', 0.7156004905700684), ('학생', 0.7085599899291992), ('고등학생', 0.6735630631446838)]


In [41]:
print(transform(model.get_nearest_neighbors(word_to_jamo('낫동생'), k=10)))

[('남동생', 0.9301387071609497), ('동생', 0.8925824761390686), ('남편', 0.797950804233551), ('남친', 0.7673526406288147), ('친구', 0.7523839473724365), ('중학생', 0.7487359046936035), ('학생', 0.7255727648735046), ('난생', 0.714683473110199), ('조카', 0.7104189395904541), ('생일', 0.708602786064148)]


In [42]:
print(transform(model.get_nearest_neighbors(word_to_jamo('납동생'), k=10)))


[('남동생', 0.916736364364624), ('동생', 0.8565492630004883), ('남편', 0.8238300085067749), ('남친', 0.799472987651825), ('난생', 0.754352331161499), ('중학생', 0.7442294359207153), ('친구', 0.7291083931922913), ('학생', 0.7184523344039917), ('남아', 0.7183816432952881), ('조카', 0.7058671116828918)]


In [43]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품질'), k=10)))

[('품질', 0.8600292801856995), ('음질', 0.822194516658783), ('땜질', 0.7546486854553223), ('찜질', 0.7204440832138062), ('퀄리티', 0.7056400179862976), ('사포질', 0.6982083916664124), ('군것질', 0.6860849857330322), ('성질', 0.6735389232635498), ('다림질', 0.649004340171814), ('퀄러티', 0.6456285715103149)]


In [44]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품쥘'), k=10)))


[('고품질', 0.8424123525619507), ('재고품', 0.7450227737426758), ('고퀄', 0.7221436500549316), ('반제품', 0.7190621495246887), ('중고품', 0.7067594528198242), ('소모품', 0.6957691311836243), ('화학제품', 0.6916146278381348), ('제품', 0.6891369223594666), ('소지품', 0.688596785068512), ('재품', 0.6871157884597778)]
